In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
from torchvision.utils import save_image
import torch.nn as nn
import torch.optim as optim

In [ ]:
class MNISTData(Dataset):
  
  # Dataset Initialization Parameters
  def __init__(self, dataset="train"):
    if dataset == "train":
      dataFile = open('sample_data/mnist_train_small.csv')
    if dataset =="test":
      dataFile = open('sample_data/mnist_test.csv')
    self.data = dataFile.readlines()
    dataFile.close()
    print(dataset,'set has ', len(self.data),' samples')
  
  # Fetching data from specific index
  def __getitem__(self, idx):
    target = torch.LongTensor([int(self.data[idx][0])])
    image = torch.Tensor([float(i) for i in self.data[idx][2:].split(',')])
    save_image(image.reshape(28,28),'img.png')
    return(image, target)
  
  # Calculating the length of the dataset
  def __len__(self):
    return(len(self.data))

# Creating DataLoaders
trainLoader = DataLoader(MNISTData("train"),batch_size=4)
testLoader = DataLoader(MNISTData("test"),batch_size=4)

train set has  20000  samples
test set has  10000  samples


In [ ]:
import torch.nn as nn


class MyNeuralNetwork(nn.Module):

# Defining Layers of the Model
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Linear(784, 400)
    self.layer2 = nn.Linear(400,100)
    self.outputLayer = nn.Linear(100, 10)


# Defining the operations regarding forward pass
  def forward(self, x):
    layer1Out = torch.sigmoid(self.layer1(x))
    layer2Out = torch.sigmoid(self.layer2(layer1Out))
    output = self.outputLayer(layer2Out)
    return (torch.softmax(output, 0))



In [ ]:
# Creating instance of the model, .cuda() ports the model to the GPU Memory
myModel = MyNeuralNetwork().cuda()
print(myModel)

# Defining Loss function and optimization algorithm
lossFn = nn.CrossEntropyLoss()
optimizer = optim.SGD(myModel.parameters(), lr = 1e-2)

# Epochs define the number of times the model will go over the entire data
for epoch in range(20):                         
  trainloss = 0.0

  # Using the dataLoader to fetch batches of training samples
  for image, target in trainLoader:
    
    image, target = image.cuda(), target.cuda()           # Porting the data to the GPU
    optimizer.zero_grad()                                 # Resetting Gradients
    output = myModel(image)                               # Forward Pass
    loss = lossFn(output, target.squeeze())               # Calculate Loss  4x10,  4
    loss.backward()                                       # Backward Pass
    optimizer.step()                                      # Weight Update
       

    #Tracking the training loss
    trainloss+=loss.item()              
  
  testloss = 0.0
  testcorrect = 0
  testtotal = 0

  # Using the dataLoader to fetch batches of test samples
  for image, target in testLoader:
    
    image, target = image.cuda(), target.cuda()           # Porting the data to the GPU
    output = myModel(image)                               # Forward Pass
    loss = lossFn(output, target.squeeze())               # Calculate Loss
    
    # Tracking the Test Loss
    testloss+=loss.item()
    
    # Calculating Accuracy
    _, predicted = torch.max(output.data, 1)              # Find model predictions for the test samples
    testtotal += target.size(0)                           # Total no. of samples
    testcorrect += (predicted == target).sum().item()     # No. of correctly classified samples
  
  # Printing results every epoch
  print('Train Loss -',trainloss/len(trainLoader),
        '- Test Loss -',testloss/len(testLoader),
        '- Test Accuracy -',100 * testcorrect // testtotal)

MyNeuralNetwork(
  (layer1): Linear(in_features=784, out_features=400, bias=True)
  (layer2): Linear(in_features=400, out_features=100, bias=True)
  (outputLayer): Linear(in_features=100, out_features=10, bias=True)
)
Train Loss - 2.1602890765190126 - Test Loss - 1.9678018528938293 - Test Accuracy - 76
Train Loss - 1.9023548152685166 - Test Loss - 1.8372609119415284 - Test Accuracy - 83
Train Loss - 1.834674150800705 - Test Loss - 1.8035554067134858 - Test Accuracy - 86
Train Loss - 1.8122262165546417 - Test Loss - 1.7888120867729187 - Test Accuracy - 87
Train Loss - 1.799916297030449 - Test Loss - 1.7788540749549866 - Test Accuracy - 89
Train Loss - 1.7920144342660904 - Test Loss - 1.7743776111602783 - Test Accuracy - 89
Train Loss - 1.7858805072784423 - Test Loss - 1.7692764681339264 - Test Accuracy - 90
Train Loss - 1.7818806226968764 - Test Loss - 1.7662573014259337 - Test Accuracy - 91
Train Loss - 1.778966247010231 - Test Loss - 1.7630080564975739 - Test Accuracy - 91
Train Loss 